In [26]:
import torch

from torch_geometric.data import Data
from torch_geometric.utils import to_dense_adj

In [45]:
edge_index = torch.tensor([
    [0, 1],
    [0, 2],
    [0, 3],
    [1, 0],
    [2, 0],
    [3, 0]
], dtype=torch.long)

x = torch.tensor([[0, 1], [0, 1], [0, 1], [0, 1]], dtype=torch.float) # configs: [0, 0, 0, 0], [1, 1, 1, 1]

y = torch.tensor([3, 1]) # ranks

data = Data(x=x, edge_index=edge_index.t().contiguous(), y=y)

In [46]:
data.num_nodes, data.num_edges, data.num_node_features, data.has_isolated_nodes(), data.has_self_loops(), data.is_directed()

(4, 6, 2, False, False, False)

In [47]:
data.keys()

['edge_index', 'y', 'x']

In [35]:
adjacency = to_dense_adj(edge_index.t().contiguous())[0]

In [36]:
adjacency

tensor([[0., 1., 1., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]])

In [48]:
data.y

tensor([3, 1])